In [1]:
print('This is for code used in the main analysis')

This is for code used in the main analysis


In [2]:
#libraries
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import plot_confusion_matrix
from sklearn.model_selection import cross_val_score

In [3]:
#import data
os.chdir("..")
data = pd.read_csv("data/bikedata.csv")
data.head()

,Unnamed: 0,TIME,NAME,BIKE.STANDS,AVAILABLE.BIKE.STANDS,AVAILABLE.BIKES,STATUS
0,1,2018-08-01 12:30:02,CHARLEMONT PLACE,40,28,12,Open
1,2,2018-08-01 12:35:02,CHARLEMONT PLACE,40,28,12,Open
2,3,2018-08-01 12:40:02,CHARLEMONT PLACE,40,28,12,Open
3,4,2018-08-01 12:45:02,CHARLEMONT PLACE,40,28,12,Open
4,5,2018-08-01 12:50:02,CHARLEMONT PLACE,40,28,12,Open


In [7]:
#Calculate bike usage
data["BIKE.USAGE"] = data["AVAILABLE.BIKE.STANDS"]/data["BIKE.STANDS"]
data.head()

NameError: name 'data' is not defined

In [ ]:
bike_dates = pd.to_datetime(data.TIME, format = "%Y-%m-%d %H:%M:%S")
print(bike_dates)

In [ ]:
bike_dates.dt.date
date_vector = pd.DatetimeIndex(bike_dates.dt.date)
print(date_vector)